In [1]:
import torch
from utils import convert_mesh_container_to_torch_mesh, cosine_similarity, double_plot, get_colors, generate_colors
from dataloaders.mesh_container import MeshContainer
from functional_map import compute_surface_map
import numpy as np

In [2]:
device = torch.device('cuda:0')
torch.cuda.set_device(device)
num_views = 100
H = 512
W = 512
num_images_per_prompt = 1
tolerance = 0.004
random_seed = 42
use_normal_map = True

In [4]:
import os
characters = os.listdir("/data/saus/saus-train-v3")
features = []
dims = []
char_names = []
for ch in sorted(characters):
    # isdir
    if not os.path.isdir(f"/data/saus/saus-train-v3/{ch}"):
        continue
    features.append(np.load(f"/data/saus/saus-train-v3/{ch}/Mesh/tpose_mesh_diff3f.npy"))
    dims.append(features[-1].shape[0])
    char_names.append(ch)
features = np.concatenate(features)
cumsum_dims = np.cumsum([0] + dims)

features = torch.from_numpy(features).float().to(device)

# PCA
mean = features.mean(dim=0)
features = features - mean
U, S, V = torch.svd(features)
features = features @ V[:, :32]

# print(np.load("/data/saus/saus-test-v2/Michelle/Mesh/Astra_walk1_subject1_8/verts0000.npy").shape)
# test_characters = os.listdir("/data/saus/saus-test-v2")
# test_characters = [ 159,  ]
test_characters = [ f for f in os.listdir("/data/shong/nrf/ASMR-Revision/") if f.endswith(".npz") ]
for ch in sorted(test_characters):
    # isdir
    # if not os.path.isdir(f"/data/saus/asmr-ood/{ch}"):
    #     continue
    os.makedirs(f"/data/saus/asmr-ood/{ch.replace('.npz', '')}/Mesh", exist_ok=True)
    feat = np.load(f"/data/shong/nrf/ASMR-Revision/{ch}")["feat"]
    np.save(f"/data/saus/asmr-ood/{ch.replace('.npz', '')}/Mesh/tpose_mesh_diff3f.npy", feat)

    feat = torch.from_numpy(feat).float().to(device)
    feat = feat - mean
    feat = feat @ V[:, :32]
    np.save(f"/data/saus/asmr-ood/{ch.replace('.npz', '')}/Mesh/tpose_mesh_diff3f_pca.npy", feat.cpu().numpy())
    # test_features.append(np.load(f"/data/saus/asmr-ood/{ch}/Mesh/tpose_mesh_diff3f.npy"))
    # test_dims.append(test_features[-1].shape[0])
    # test_char_names.append(ch)
# test_features = np.concatenate(test_features)
# test_cumsum_dims = np.cumsum([0] + test_dims)

# test_features = torch.from_numpy(test_features).float().to(device)
# test_features = test_features - mean
# test_features = test_features @ V[:, :32]

# np.save(f"/data/saus/asmr-ood/{test_characters[0]}/Mesh/tpose_mesh_diff3f_pca.npy", test_features.cpu().numpy())
# print(test_features.shape)

In [32]:
src_idx = np.random.randint(0, len(dims))
src_mesh = np.load(f"/data/saus/saus-train-v3/{char_names[src_idx]}/Mesh/tpose_mesh.npz")
src_mesh = MeshContainer(src_mesh["c_rest_pos"], src_mesh["fid_to_cids"])

tgt_idx = np.random.randint(0, len(dims))
tgt_mesh = np.load(f"/data/saus/saus-train-v3/{char_names[tgt_idx]}/Mesh/tpose_mesh.npz")
tgt_mesh = MeshContainer(tgt_mesh["c_rest_pos"], tgt_mesh["fid_to_cids"])

# original features before PCA compression
src_feature_orig = torch.from_numpy(np.load(f"/data/saus/saus-train-v3/{char_names[src_idx]}/Mesh/tpose_mesh_diff3f.npy")).float().to(device)
tgt_feature_orig = torch.from_numpy(np.load(f"/data/saus/saus-train-v3/{char_names[tgt_idx]}/Mesh/tpose_mesh_diff3f.npy")).float().to(device)
s_orig = cosine_similarity(src_feature_orig, tgt_feature_orig)
s_orig = torch.argmax(s_orig, dim=0).cpu().numpy()
cmap_source_orig = get_colors(src_mesh.vert)
cmap_target_orig = cmap_source_orig[s_orig]

double_plot(src_mesh, tgt_mesh, cmap_source_orig, cmap_target_orig)

# compressed features
feature_pca = []
for i in range(len(dims)):
    feature_pca.append(features[cumsum_dims[i]:cumsum_dims[i+1]])
src_feature_pca = feature_pca[src_idx]
tgt_feature_pca = feature_pca[tgt_idx]

s = cosine_similarity(src_feature_pca.to(device), tgt_feature_pca.to(device))
s = torch.argmax(s, dim=0).cpu().numpy()
cmap_source = get_colors(src_mesh.vert)
cmap_target = cmap_source[s]

double_plot(src_mesh, tgt_mesh,cmap_source, cmap_target)  


In [33]:
src_idx = np.random.randint(0, len(dims))
src_mesh = np.load(f"/data/saus/saus-train-v3/{char_names[src_idx]}/Mesh/tpose_mesh.npz")
src_mesh = MeshContainer(src_mesh["c_rest_pos"], src_mesh["fid_to_cids"])

tgt_idx = np.random.randint(0, len(test_dims))
tgt_mesh = np.load(f"/data/saus/saus-test-v3/{test_char_names[tgt_idx]}/Mesh/tpose_mesh.npz")
tgt_mesh = MeshContainer(tgt_mesh["c_rest_pos"], tgt_mesh["fid_to_cids"])

# original features before PCA compression
src_feature_orig = torch.from_numpy(np.load(f"/data/saus/saus-train-v3/{char_names[src_idx]}/Mesh/tpose_mesh_diff3f.npy")).float().to(device)
tgt_feature_orig = torch.from_numpy(np.load(f"/data/saus/saus-test-v3/{test_char_names[tgt_idx]}/Mesh/tpose_mesh_diff3f.npy")).float().to(device)
s_orig = cosine_similarity(src_feature_orig, tgt_feature_orig)
s_orig = torch.argmax(s_orig, dim=0).cpu().numpy()
cmap_source_orig = get_colors(src_mesh.vert)
cmap_target_orig = cmap_source_orig[s_orig]

double_plot(src_mesh, tgt_mesh, cmap_source_orig, cmap_target_orig)

# compressed features
test_feature_pca = []
for i in range(len(test_dims)):
    test_feature_pca.append(test_features[test_cumsum_dims[i]:test_cumsum_dims[i+1]])
src_feature_pca = feature_pca[src_idx]
tgt_feature_pca = test_feature_pca[tgt_idx]

s = cosine_similarity(src_feature_pca.to(device), tgt_feature_pca.to(device))
s = torch.argmax(s, dim=0).cpu().numpy()
cmap_source = get_colors(src_mesh.vert)
cmap_target = cmap_source[s]

double_plot(src_mesh, tgt_mesh,cmap_source, cmap_target)

In [34]:
# save
for i in range(len(dims)):
    np.save(f"/data/saus/saus-train-v3/{char_names[i]}/Mesh/tpose_mesh_diff3f_pca.npy", feature_pca[i].cpu().numpy())
    print(f"saved {char_names[i]} shape {feature_pca[i].shape}")
for i in range(len(test_dims)):
    np.save(f"/data/saus/saus-test-v3/{test_char_names[i]}/Mesh/tpose_mesh_diff3f_pca.npy", test_feature_pca[i].cpu().numpy())
    print(f"saved {test_char_names[i]} shape {test_feature_pca[i].shape}")

saved AlienSoldier shape torch.Size([4783, 32])
saved Amy shape torch.Size([4826, 32])
saved Brady shape torch.Size([4906, 32])
saved Brian shape torch.Size([4985, 32])
saved Bryce shape torch.Size([4866, 32])
saved Chad shape torch.Size([4898, 32])
saved Claire shape torch.Size([4224, 32])
saved CopzombieL shape torch.Size([4821, 32])
saved Jolleen shape torch.Size([4693, 32])
saved Kaya shape torch.Size([4260, 32])
saved MariaJJOng shape torch.Size([4780, 32])
saved Mousey shape torch.Size([4950, 32])
saved PeasantMan shape torch.Size([2414, 32])
saved PumpkinhulkL shape torch.Size([4858, 32])
saved TheBoss shape torch.Size([4799, 32])
saved Timmy shape torch.Size([4931, 32])
saved VanguardByTChoonyung shape torch.Size([4772, 32])
saved Astra shape torch.Size([4728, 32])
saved CastleGuard01 shape torch.Size([2410, 32])
saved Doozy shape torch.Size([4967, 32])
saved Jackie shape torch.Size([4754, 32])
saved Michelle shape torch.Size([4974, 32])
saved MremirehODesbiens shape torch.Size